In [2]:
import pandas as pd
import pymorphy3
import re
import swifter

In [2]:
def string_dev (a_in):                              # Очистка текта от тегов и специальных символов
    x_in = re.sub(r'\n',' ',a_in)                    # удаление символов конца строки
    y_in = x_in.lower()                             # приведение ситоки к нижнему регистру
    b_in = re.sub(r'&\w+;',' ', y_in)                 # замена тегов типа'&nbsp;'  на пробел
    d_in = re.sub(r'<[^>]*>',' ', b_in)               # удаление тегов
    f_in = re.sub(r'www.\w+.\w{2,3}?',' ', d_in)      # удаление адресов веб-серверов
#\xad
#–\xa0
    a_in = re.sub(r'\xad','', f_in)
    c_in = re.sub(r'\\xa0-','', a_in)
    u_in = re.sub(r'\\u200e','', c_in)    
    w_in = re.sub(r'\d+','', u_in)                        # удаление последовательностей цифр
    y_in = re.sub(r'[\-]',' ',w_in)
    yy_in = re.sub(r'[_]+', ' ',y_in)
    yyy_in = re.sub(r'[ι,…,—,–,//,\(,\),",\[,\],\\\\,\\,,\-,:,;,<,>,=,©,@,№,#,%,\',\+,\*,“,”,&,∙,~,\$,\^,•,«,»,_,ι,і,‑,‘,’,і]+','',yy_in)
#    s_in = re.sub(r'\W', ' ', y_in)
    q_in = re.sub(r'[a-z]*','', yyy_in)                 # удаление латинских букв
    
    sss_in = re.sub(r'\b\w{,2}\b','', q_in)          # удаление всех слов длиной до 2-х букв
    qms_in = re.sub(r'\s{2,}',' ', q_in)            # замена кратного числа пробелов на один
    nms_in = qms_in.strip()
#    nms_in = re.sub(r'^[^\w]*\s', '', qms_in)         # удаление пробелов в начале строки
#    mms_in = re.sub(r'\s*$','', nms_in)              # удаление пробелов в конце строки
    return (nms_in)

In [3]:
def lematization (f_input_list):          # Лематизация слов в списке
    morph = pymorphy3.MorphAnalyzer()
    lnorm = list()
    for word in f_input_list:
        p = morph.parse(word)[0]
        lnorm.append(p.normal_form)
    return (lnorm)

In [4]:
def del_my_stop_words(word_tokens): # удаление из списка моих стоп-слов
    stop_words_nltk = {'который', 'кому', 'имя', 'сегодня', 'вчера', 'завтра', 'также', 'в', 'во', 'свой', 
                  'это', 'часто', 'зачастую', 'мочь', 'смочь','а', 'без', 'более', 'больше', 'будет', 'будто', 
                  'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот', 
                  'впрочем', 'все', 'всегда', 'всего', 'всех', 'всю', 'вы', 'где', 'да', 'даже', 'два', 'для', 
                  'до', 'другой', 'его', 'ее', 'ей', 'ему', 'если', 'есть', 'еще', 'ж', 'же', 'за', 'зачем', 'здесь', 
                  'и', 'из', 'или', 'им', 'иногда', 'их', 'к', 'как', 'какая', 'какой', 'когда', 'конечно', 'кто', 
                  'куда', 'ли', 'лучше', 'между', 'меня', 'мне', 'много', 'может', 'можно', 'мой', 'моя', 'мы', 'на',
                  'над', 'надо', 'наконец', 'нас', 'не', 'него', 'нее', 'ней', 'нельзя', 'нет', 'ни', 'нибудь', 
                  'никогда', 'ним', 'них', 'ничего', 'но', 'ну', 'о', 'об', 'один', 'он', 'она', 'они', 'опять', 
                  'от', 'перед', 'по', 'под', 'после', 'потом', 'потому', 'почти', 'при', 'про', 'раз', 'разве', 
                  'с', 'со', 'сам', 'свою', 'себе', 'себя', 'сейчас', 'c', 'со', 'совсем', 'так', 'такой', 'там', 'тебя', 
                  'тем', 'теперь', 'то', 'тогда', 'того', 'тоже', 'только', 'том', 'тот', 'три', 'тут', 'ты', 
                  'у', 'уж', 'уже', 'хорошо', 'хоть', 'что', 'чего', 'чем', 'через', 'что', 'чтоб', 'чтобы', 'чуть', 
                  'эти', 'этого', 'этой', 'этом', 'этот', 'эту', 'я', 'сказал', 'человек', 'жизнь', 'говорил', 'кажется', 
                  'сказать', 'сегодня', 'сказала', 'сказал'} 

    my_stop_words = {'сгт','свой', 'стать', 'кроме', 'разный', 'около', 'затем', 'помимо', 'ваш', 'вам', 'некоторый', 
                     'лишь', 'каждый', 'самый', 'также', 'неоднократно', 'ещё', 'сразу', 'среди',
                  'однако', 'вновь', 'иной', 'ныне', 'пока', 'хотя','либо','немного', 'гораздо', 'ничто', 'нередко', 'ныне', 
                  'наоборот', 'впереди', 'таковой', 'мимо', 'тесно', 'вряд', 'нечто', 'почём', 'почему', 'любой', 'обратно',
                  'оттуда', 'очень', 'понапрасну', 'поскольку', 'почему', 'поэтому', 'прежде', 'причём', 'прочий', 'пусть', 
                  'пока', 'это', 'наш', 'несколько', 'около', 'помимо', 'однако', 'сколько', 'либо', 'гораздо', 'ничто',
                   'наоборот', 'никак', 'таковой', 'твой', 'нечто', 'понапрасну', 'почём', 'подробный', 'информация'}

    stop_words = stop_words_nltk | my_stop_words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    return filtered_sentence

In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)


def remove_proper_nouns(text):
    # Пропускаем не-строки и пустые строки
    if not isinstance(text, str) or not text.strip():
        print("text имеет неожидаемый тип")
        return text
    
    try:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.tag_ner(ner_tagger)
        
        # Если нет сущностей, возвращаем исходный текст
        if not doc.spans:
            return text
        
        # Собираем сущности для удаления
        spans_to_remove = [span for span in doc.spans if span.type in ['PER', 'LOC', 'ORG']]
        
        # Если нечего удалять
        if not spans_to_remove:
            return text
            
        # Удаляем сущности (начиная с конца)
        text_cleaned = text
        for span in sorted(spans_to_remove, key=lambda x: x.start, reverse=True):
            text_cleaned = text_cleaned[:span.start] + text_cleaned[span.stop:]
            
        return text_cleaned.strip()
    
    except Exception as e:
        print(f"Ошибка при обработке текста: {text[:50]}... Ошибка: {str(e)}")
        return text  # В случае ошибки возвращаем исходный текст

In [6]:
def process_str_of_the_news(string):  # Обработка сырой строки-новости 
    global abbr_list
    string_2 = string_dev(string)     # Предобработка строки
    
    if len(string_2) != 0:  # Если строка непустая после обработки
        list_of_sentences = re.split(r'[.!?]', string_2)
        list_of_sentences = [x.split(' ') for x in list_of_sentences if x]
        
        filtered_sentences = []
        for sentence in list_of_sentences:
            no_stopwords_1 = del_my_stop_words(sentence)
            lemmatized = lematization(no_stopwords_1)
            no_stopwords_2 = del_my_stop_words(lemmatized)
            text = ' '.join(no_stopwords_2)
            # Удаление слов длиной ≤ 2 символов
            text = re.sub(r'\b\w{1,2}\b', '', text)
            # Замена нескольких пробелов на один
            text = re.sub(r'\s{2,}', ' ', text)
            text = text.strip()
            if text and len(text) > 0:
                filtered_sentences.append(text)
    
    return filtered_sentences


In [ ]:
def apply_tokenization():
    docs = pd.read_parquet("raw-data.pq")
    docs['News_Tokens'] = docs["News_Text"].swifter.apply(process_str_of_the_news)
    return docs
docs = apply_tokenization()

c:\Users\infer\AppData\Local\Programs\Python\Python312\Lib\site-packages\bodo\transforms\distributed_analysis.py:305: BodoWarning: No parallelism found for function 'apply_tokenization'. Distributed diagnostics:
Distributed analysis set ''docs'' as replicated due to call to function 'pandas.read_excel' (unsupported function or usage)

File "..\..\..\AppData\Local\Temp\ipykernel_9588\3947763556.py", line 3:
<source missing, REPL/exec in use?>
  warnings.warn(


In [ ]:
docs.to_csv("data_1.csv", index=False)

In [ ]:
docs['News_Tokens'] = docs['News_Tokens'].apply(
    lambda sentence_list: [remove_proper_nouns(sent) for sent in sentence_list]
)
docs.to_csv("data.csv", index=False)

In [3]:
docs = pd.read_excel("News_SGU_31077_Processed_1.xlsx")
docs.to_parquet("raw-data.pq")